# Lectura de ficheros easyMoney

## import Librariesz

In [1]:
import numpy as np 
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import matplotlib as plt


In [2]:
ca_df = pd.read_parquet('https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/commercial_activity_df.parquet')

prod_df = pd.read_parquet('https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/products_df.parquet')

sdg_df = pd.read_parquet('https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/sociodemographic_df.parquet')

In [3]:
ca_df

,Unnamed: 0,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,0,1375586,2018-01-28,2018-01-12,KHL,1.00,02 - PARTICULARES
1,1,1050611,2018-01-28,2015-08-10,KHE,0.00,03 - UNIVERSITARIO
2,2,1050612,2018-01-28,2015-08-10,KHE,0.00,03 - UNIVERSITARIO
3,3,1050613,2018-01-28,2015-08-10,KHD,0.00,03 - UNIVERSITARIO
4,4,1050614,2018-01-28,2015-08-10,KHE,1.00,03 - UNIVERSITARIO
...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,2016-08-14,KHE,0.00,03 - UNIVERSITARIO
5962920,13647305,1166764,2019-05-28,2016-08-14,KHE,0.00,03 - UNIVERSITARIO
5962921,13647306,1166763,2019-05-28,2016-08-14,KHE,1.00,02 - PARTICULARES
5962922,13647307,1166789,2019-05-28,2016-08-14,KHE,0.00,03 - UNIVERSITARIO


In [4]:
prod_df

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
3,3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0
4,4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
5962920,13647305,1166764,2019-05-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
5962921,13647306,1166763,2019-05-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1
5962922,13647307,1166789,2019-05-28,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1


In [6]:
sdg_df

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.00,H,35,N,"87,218.10"
1,1,1050611,2018-01-28,ES,13.00,V,23,N,"35,548.74"
2,2,1050612,2018-01-28,ES,13.00,V,23,N,"122,179.11"
3,3,1050613,2018-01-28,ES,50.00,H,22,N,"119,775.54"
4,4,1050614,2018-01-28,ES,50.00,V,23,N,NaN
...,...,...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,ES,50.00,V,22,N,"43,912.17"
5962920,13647305,1166764,2019-05-28,ES,26.00,V,23,N,"23,334.99"
5962921,13647306,1166763,2019-05-28,ES,50.00,H,47,N,NaN
5962922,13647307,1166789,2019-05-28,ES,50.00,H,22,N,"199,592.82"
